In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df1_WS4T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/pythonProject/segment1_WS4_has_duration.csv")

/Users/asteriag/opt/anaconda3/envs/pythonProject/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df1_WS4T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,1079339|441319|682219,Release Offer,Release Offer,11/5/2019 14:07:56,ATKINSONM,Faculty,16,N,3b. Offer stage,Part 2 - Application Line ID Workflow,...,MCW,HDCW,Full Degree,OF,OF,Y,$DEF,Y,46 days 23 hours 34 minutes 3 seconds,4059243.0


In [4]:
numOfWorkflowProcesses = df1_WS4T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic


In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [9]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df1_WS4T[df1_WS4T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Accept','Defer'])]
        count_df_id += df_id['CASE_ID'].count()
        count_df_end_step += df_end_step['CASE_ID'].count()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [10]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df1_WS4T[df1_WS4T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [11]:
s1_t4=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s1_t4.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step
0,1,1698,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Release Offer}
1,2,2183,63 days 6 hours 38 minutes 47 seconds,33.0 days 3.0 hours 22.0 minutes 13.0 seconds,36.6%,"{OC - Reviewed - Other, Decline, Assessment De..."
2,3,1108,73 days 23 hours 29 minutes 16 seconds,47.0 days 1.0 hours 43.0 minutes 8.0 seconds,25.3%,"{OC - Reviewed - Other, Decline, AR - Reviewed..."
3,4,608,83 days 21 hours 14 minutes 28 seconds,64.0 days 11.0 hours 34.0 minutes 57.0 seconds,20.11%,"{OC - Reviewed - Other, Decline, AR - Reviewed..."
4,5,385,104 days 10 hours 6 minutes 16 seconds,86.0 days 1.0 hours 59.0 minutes 42.0 seconds,17.56%,"{OC - Reviewed - Other, Decline, Acacdemic - A..."
5,6,202,109 days 6 hours 59 minutes 52 seconds,89.0 days 23.0 hours 3.0 minutes 43.0 seconds,14.36%,"{OC - Reviewed - Other, Decline, AR - Reviewed..."
6,7,126,120 days 6 hours 32 minutes 2 seconds,104.0 days 16.0 hours 22.0 minutes 7.0 seconds,12.59%,"{OC - Reviewed - Other, Decline, Back to Triag..."
7,8,72,124 days 11 hours 16 minutes 41 seconds,120.0 days 12.0 hours 11.0 minutes 27.5 seconds,11.81%,"{OC - Reviewed - Other, Decline, AR - Reviewed..."
8,9,36,121 days 9 hours 45 minutes 36 seconds,99.0 days 8.0 hours 50.0 minutes 33.0 seconds,9.88%,"{OC - Reviewed - Other, Decline, Manage Accept..."
9,10,11,241 days 8 hours 21 minutes 46 seconds,246.0 days 4.0 hours 58.0 minutes 4.0 seconds,9.09%,"{Manage Acceptance, AR - Satisfied - Other, OC..."


In [12]:
s1_t4.to_csv('segment1_WS4_table.csv', index=False)





